In [1]:
#Importing libraries
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [2]:
#Loading the csv files
df=pd.read_csv("./datasets/Movies/Train.csv")

In [3]:
#Converting dataframes-->numpy array
data=df.values

In [4]:
#Splitting X and Y data
X=data[:,:1]
Y=data[:,1,]

In [8]:
#Initialising out TOkenizer,stopwords and Stemmer
rg=RegexpTokenizer(r'[a-z@._]+')
en_stopwords=set(stopwords.words('english'))
ps=PorterStemmer();

In [9]:
#Converting document into bag of words
def getstemmedreview(X):
    x_clean=[]
    for review in X:
        review=str(review).lower()
        review=review.replace("<br /><br />"," ")
        tokens=rg.tokenize(review)
        new_tokens=[token for token in tokens if token not in en_stopwords or token=='not']
        stemmed_tokens=[ps.stem(token) for token in new_tokens]
        sent=' '.join(stemmed_tokens)
        x_clean.append(sent)
    print(len(x_clean))
    return x_clean

In [10]:
x=getstemmedreview(X)

40000


In [11]:
#Encoding the values
Y=Y.reshape(-1,)
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit(Y)
y=le.transform(Y)

In [12]:
#Creating personalised vocab as we have only two labels so I am using vocab_p and vocab_b to save computations
def vocab(x):
    vocabulary_p={}
    vocabulary_n={}
    for i in range(len(x)):
        reviews=x[i]
        review=reviews.split(' ')
        if y[i]==1:
            for words in review:
                if str(words) in vocabulary_p:
                    vocabulary_p[words]+=1
                else:
                    vocabulary_p[words]=1
        else:
            for words in review:
                if words in vocabulary_n:
                    vocabulary_n[words]+=1
                else:
                    vocabulary_n[words]=1
    return vocabulary_p,vocabulary_n;

In [13]:
dictionary_p,dictionary_n=vocab(x)

In [14]:
#Loading testing data
x_test=pd.read_csv('./datasets/Movies/Test.csv')
X_test=x_test.values

In [15]:
#Calculating Conditional Prob and using laplace smoothing
def cond_prob(x_train,feature_value,label):
    num=1
    denom=len(dictionary_p)
    if feature_value in dictionary_p:
        denom+=dictionary_p[str(feature_value)]
    if feature_value in dictionary_n:   
        dictionary_n[str(feature_value)]
    if label==1 and feature_value in dictionary_p:
        num=dictionary_p[str(feature_value)]
    elif feature_value in dictionary_n:
        num=dictionary_n[str(feature_value)]
    
    return num/denom

In [16]:
 x_test=getstemmedreview(X_test)

10000


In [17]:
#Will handle prediction and save all the vlaues in y_pred
def predict(x_train,y_train,x_test):
    labels=np.unique(y_train)
    y_pred=[]
    for doc in x_test:
        pred=[]
        for label in labels:
            likelihood=1.0
            docx=doc.split(' ')
            for feature in docx:
                likelihood*=cond_prob(x_train,str(feature),label)
            pred.append(likelihood)
        y_pred.append(np.argmax(pred))
    return y_pred
            

In [18]:
y_pred=predict(x,y,x_test)

In [19]:
#Used to Store output
def output(y_pred):
    ans='Id,label\n'
    for i in range(len(y_pred)):
        ans+=(str(i)+','+ ('pos' if y_pred[i]==1 else 'neg')+'\n')
    with open('movie.csv','w') as f:
        f.write(ans)

In [27]:
#Trying Using Sklearn and TfIDf normalisation
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
#Creating objects for the classes
mnb=MultinomialNB()
tfidf=TfidfVectorizer()

In [22]:
#Fir_transform the training data and transform the testing data as it will keep the shape intact
x_vec=tfidf.fit_transform(x)
x_vec_test=tfidf.transform(x_test)

In [23]:
mnb.fit(x_vec,y)

MultinomialNB()

In [24]:
mnb.score(x_vec,y)

0.90125

In [25]:
y_pred=mnb.predict(x_vec_test)

In [26]:
#Saving output to a file
output(y_pred)